In [0]:
# test 

import numpy as np
# Example dummy data from Rendle 2010 
# http://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf
# Variables categoricas (Users, Wines, Last Rated) pasaron por one-hot-encoding 

x_data = np.matrix([
#     Users  |     Wines      |    Wine Ratings    | Time | Last Wines Rated
#    A  B  C | W1  W2  W3  W4 | W1   W2   W3   W4  |      | W1  W2  W3  W3
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])

# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])

# Agregamos un eje para que funcione Tensorflow.
y_data.shape += (1, )

import tensorflow as tf
n, p = x_data.shape

# numero de latent factors
k = 5

# design matrix
X = tf.placeholder('float', shape=[n, p])
# target vector
y = tf.placeholder('float', shape=[n, 1])

# bias y weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))

# interaction factors, inicializados de forma aleatoria 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))

# estimación de y, inicializada en 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

Usamos placeholders para los inputs y targets. Los datos actuales se asignarán en el runtime de la sesión. X e Y no serán modificados por el backend; Usamos variables para almacenar bias, weights y factor layers. Estos son los parámetros que se actualizarán al ajustar el modelo.

En el siguiente código, computamos WX y usamos reduce_sum() para agregar las filas del Tensor resultante (axis 1). keep_dims está seteado en True para asegurar que las dimensiones input/output se respeten.

In [0]:
linear_terms = tf.add(w0,
        	tf.reduce_sum(
          tf.multiply(W, X), 1, keepdims=True))

En chunk anterior implementamos simplemente una regresion lineal. Hacemos lo mismo con los términos de interacción.

In [4]:
interactions = (tf.multiply(0.5,
                tf.reduce_sum(
                    tf.subtract(
                        tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                        tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),
                    1, keep_dims=True)))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


Y agregamos todo para obtener el target estimado.

In [0]:
y_hat = tf.add(linear_terms, interactions)

Como estamos resolviendo un problema de regresion, aprendemos los parámetros del modelo minimizando la suma de los residuos cuadráticos como función de pérdida. También agregamos un término de regularización para prevenir overfitting.

In [0]:
# Función de pérdida como suma de cuadrados regularizada L2  sobre W y V
lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = (tf.reduce_sum(
            tf.add(
                tf.multiply(lambda_w, tf.pow(W, 2)),
                tf.multiply(lambda_v, tf.pow(V, 2)))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

Para entrenar el modelo declamos un optimizador y minimizamos la función de pérdida.

In [0]:
eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

Estamos listos para obtener el grafo y desplegarlo en el backend de Tensorflow. Usamos pythos context manager pata manejar la sesion.

In [9]:
# Muchas Iteraciones
N_EPOCHS = 1000

# Desplegar grafo
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(N_EPOCHS):
        indices = np.arange(n)
        np.random.shuffle(indices)
        x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

    print('MSE: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
    print('Loss (regularized error):', sess.run(loss, feed_dict={X: x_data, y: y_data}))
    print('Predictions:', sess.run(y_hat, feed_dict={X: x_data, y: y_data}))
    print('Learnt weights:', sess.run(W, feed_dict={X: x_data, y: y_data}))
    print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))

MSE:  0.009058842
Loss (regularized error): 0.012352985
Predictions: [[4.9134507 ]
 [0.8834113 ]
 [2.8877802 ]
 [0.94780743]
 [4.899725  ]
 [4.8810534 ]
 [3.9470143 ]]
Learnt weights: [ 0.1073296   0.19500588 -0.05508566  0.01161111 -0.04774291  0.19293357
  0.1013393   0.01571284  0.06099707  0.09444442  0.14801686  0.108797
  0.15124144 -0.15934798  0.1013393   0.        ]
Learnt factors: [[ 0.18526642  0.10820723 -0.2842518   0.15335539 -0.46750578  0.35297954
  -0.0352198  -0.03085277  0.17319572  0.03563618  0.09997714  0.28912053
   0.29515004 -0.54035366 -0.0388333  -0.00266565]
 [ 0.0598817   0.2097321  -0.06389549  0.06335289 -0.159418    0.13162969
   0.07834444 -0.00188725  0.03313552  0.02868493  0.18614493  0.18674748
   0.14345922 -0.33916256  0.08197381 -0.00210224]
 [ 0.03983677  0.15426078 -0.11922389  0.04991257 -0.12058035  0.07039631
   0.07574358 -0.02698277 -0.01795151  0.01485645  0.12877299  0.10219675
   0.04944892 -0.26571015  0.06601847 -0.00940343]
 [ 0.0696

En cada iteración (hasta N_EPOCHS) ejecutamos el optimizador, el cual recalcula los parametros del modelo por el método de descenso del gradiente. Notar que estamos moviendo datos desde el espacio de memoria de Python al de backend C++ de Tensorflow  a través de feed_dict={}.
Como estamos trabajando con un set reducido de datos exprimentales podemos entregar el dataset todo de una vez. En la práctica, querremos trabajar con mini batches (ej. usando un generador sobre el input). Podemos mezclar los datos (np.random.shuffle) para evitar sesgar el gradiente.

# Experimentos

Para setear un benchmark  del performance en general del modelo, entrenamos y testeamos en el dataset porcionado en training y test (70%-30%). El objetivo es predecir el rating para cada vino. Luego del aplicar one-hot-encoding a las variables categóricas, obtendemos una design matrix de 90570×2623 [Corregir] para el training set, y de 90570×2623[Corregir] para el test set.


## Representación para el training

Obtenemos un subset pequeño del dataset desde http://www.cellartracker.com, obtenemos los archivos ........

La forma más facil de utilizar esta clase es representando la training data como listas de Python dict objects, donde los dict elements mapean las variables categóricas y numericas de cada intancia a sus valores. Luego usamos DictVectorizer de sklearn  para convertirlas en una disign matrix encodeada con one-of-K or “one-hot” coding.

Un pequeño ejemplo:

In [11]:
import numpy as np
import pandas as pd

from google.colab import files
uploaded = files.upload()


Saving Reviews.csv to Reviews (1).csv


In [14]:
reviews = pd.read_csv('Reviews.csv',
                         sep=';',
                         names = ['userID','review', 'review_URL','score','sentiment','date', 
                                  'vintage', 'type', 'producer', 'variety', 'designation', 'vineyard', 
                                  'country', 'region', 'subregion', 'appellation', 'price_level'],
                         header=0)


print("Matrix size",reviews.shape)

# view columns using df.columns
print("Column Names:",reviews.columns)

print(reviews.dtypes)


#reviews.head()
reviews.tail()


Matrix size (20000, 17)
Column Names: Index(['userID', 'review', 'review_URL', 'score', 'sentiment', 'date',
       'vintage', 'type', 'producer', 'variety', 'designation', 'vineyard',
       'country', 'region', 'subregion', 'appellation', 'price_level'],
      dtype='object')
userID          object
review          object
review_URL      object
score          float64
sentiment       object
date            object
vintage        float64
type            object
producer        object
variety         object
designation     object
vineyard        object
country         object
region          object
subregion       object
appellation     object
price_level      int64
dtype: object


,userID,review,review_URL,score,sentiment,date,vintage,type,producer,variety,designation,vineyard,country,region,subregion,appellation,price_level
19995,Anonymous,Very sweet extracted wine. This was drinakable...,https://www.cellartracker.com/notes.asp?iWine=...,82.0,Anonymous wrote:,11/27/2006 - \nAnonymous wrote: \n\n82 Points,2004.0,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1
19996,msinger,This wine was nothing special - a pizza wine f...,https://www.cellartracker.com/notes.asp?iWine=...,NaN,msinger wrote:,6/28/2006 - \nmsinger wrote:,2004.0,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1
19997,jrglm,Yuck,https://www.cellartracker.com/notes.asp?iWine=...,75.0,jrglm wrote:,3/22/2006 - \njrglm wrote: \n\n75 Points,2004.0,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1
19998,bottlestopper,yuck! Old and well past its best,https://www.cellartracker.com/notes.asp?iWine=...,NaN,bottlestopper wrote:,7/10/2011 - \nbottlestopper wrote:,2003.0,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1
19999,Slipknot,whatever,https://www.cellartracker.com/notes.asp?iWine=...,80.0,Slipknot wrote:,11/15/2006 - \nSlipknot wrote: \n\n80 Points,2003.0,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1


In [16]:
# categorical data using df.select_dtypes() La idea es poner una dummy por cada categoria
reviews_categorical = reviews.select_dtypes(include=[object])

# view columns using df.columns
print("Column Names:",reviews_categorical.columns)

reviews['userID'] = reviews['userID'].astype('str')
print(reviews.dtypes)

reviews_categorical.tail(3)

Column Names: Index(['userID', 'review', 'review_URL', 'sentiment', 'date', 'type',
       'producer', 'variety', 'designation', 'vineyard', 'country', 'region',
       'subregion', 'appellation'],
      dtype='object')
userID          object
review          object
review_URL      object
score          float64
sentiment       object
date            object
vintage        float64
type            object
producer        object
variety         object
designation     object
vineyard        object
country         object
region          object
subregion       object
appellation     object
price_level      int64
dtype: object


,userID,review,review_URL,sentiment,date,type,producer,variety,designation,vineyard,country,region,subregion,appellation
19997,jrglm,Yuck,https://www.cellartracker.com/notes.asp?iWine=...,jrglm wrote:,3/22/2006 - \njrglm wrote: \n\n75 Points,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura
19998,bottlestopper,yuck! Old and well past its best,https://www.cellartracker.com/notes.asp?iWine=...,bottlestopper wrote:,7/10/2011 - \nbottlestopper wrote:,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura
19999,Slipknot,whatever,https://www.cellartracker.com/notes.asp?iWine=...,Slipknot wrote:,11/15/2006 - \nSlipknot wrote: \n\n80 Points,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura


In [0]:
reviews['vintage'] = reviews['vintage'].fillna(0).astype('int')

print(reviews.dtypes)

print("Matrix size",reviews.shape)

# import preprocessing from sklearn
from sklearn import preprocessing

# view columns using df.columns
print("Column Names:",reviews.columns)
print(type(reviews))

#reviews = reviews.Reviews.unique()
#print("Unique items Matrix size",reviews.shape)


userID           object
review           object
review_URL       object
score           float64
sentiment        object
sate             object
vintage           int64
type             object
producer         object
variety          object
designation      object
vineyard         object
country          object
region           object
subregion        object
appellation      object
Pprice_level      int64
dtype: object
Matrix size (20000, 17)
Column Names: Index(['userID', 'review', 'review_URL', 'score', 'sentiment', 'sate',
       'vintage', 'type', 'producer', 'variety', 'designation', 'vineyard',
       'country', 'region', 'subregion', 'appellation', 'Pprice_level'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>


Nuestro review df variable contiene un pandas dataframe con 20800 reviews rows y 17 columns con wines features. Todas excepto score, year y vintage son categorical features which will need to be transformed in order to use as inputs in the various machine learning algorithms. Let’s now perform one hot encoding on these categorical variables.

In [0]:
from sklearn.preprocessing import LabelEncoder

le_color = LabelEncoder()
le_make = LabelEncoder()

reviews['type_encoded'] = le_color.fit_transform(reviews.type)
reviews['producer_encoded'] = le_color.fit_transform(reviews.producer)
reviews['variety_encoded'] = le_color.fit_transform(reviews.variety)

print(reviews.dtypes)

reviews.tail(3)


userID               object
review               object
review_URL           object
score               float64
sentiment            object
sate                 object
vintage               int64
type                 object
producer             object
variety              object
designation          object
vineyard             object
country              object
region               object
subregion            object
appellation          object
Pprice_level          int64
type_encoded          int64
producer_encoded      int64
variety_encoded       int64
dtype: object


,userID,review,review_URL,score,sentiment,sate,vintage,type,producer,variety,designation,vineyard,country,region,subregion,appellation,Pprice_level,type_encoded,producer_encoded,variety_encoded
19997,jrglm,Yuck,https://www.cellartracker.com/notes.asp?iWine=...,75.0,jrglm wrote:,3/22/2006 - \njrglm wrote: \n\n75 Points,2004,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1,2,1041,188
19998,bottlestopper,yuck! Old and well past its best,https://www.cellartracker.com/notes.asp?iWine=...,NaN,bottlestopper wrote:,7/10/2011 - \nbottlestopper wrote:,2003,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1,2,1041,188
19999,Slipknot,whatever,https://www.cellartracker.com/notes.asp?iWine=...,80.0,Slipknot wrote:,11/15/2006 - \nSlipknot wrote: \n\n80 Points,2003,Red,Mad Dogs & English Men,Red Blend,NaN,NaN,Spain,Extremadura,NaN,Vino de la Tierra Extremadura,1,2,1041,188


In [0]:

from sklearn.preprocessing import OneHotEncoder

type_ohe = OneHotEncoder()
producer_ohe = OneHotEncoder()
variety_ohe = OneHotEncoder()
designation_ohe = OneHotEncoder()
vineyard_ohe = OneHotEncoder()
country_ohe = OneHotEncoder()
region_ohe = OneHotEncoder()
subregion_ohe = OneHotEncoder()
appellation_ohe = OneHotEncoder()

type_ = type_ohe.fit_transform(reviews.type_encoded.values.reshape(-1,1)).toarray()
producer_ = producer_ohe.fit_transform(reviews.producer_encoded.values.reshape(-1,1)).toarray()
variety_ = variety_ohe.fit_transform(reviews.variety_encoded.values.reshape(-1,1)).toarray()

typeOneHot = pd.DataFrame(type_, columns = ["Producer_"+str(int(i)) for i in range(type_.shape[1])])
producerOneHot = pd.DataFrame(producer_, columns = ["Producer_"+str(int(i)) for i in range(producer_.shape[1])])


df = pd.concat([reviews, typeOneHot,producerOneHot], axis=1)

df.tail(3)

,userID,review,review_URL,score,sentiment,sate,vintage,type,producer,variety,...,Producer_1208,Producer_1209,Producer_1210,Producer_1211,Producer_1212,Producer_1213,Producer_1214,Producer_1215,Producer_1216,Producer_1217
19997,jrglm,Yuck,https://www.cellartracker.com/notes.asp?iWine=...,75.0,jrglm wrote:,3/22/2006 - \njrglm wrote: \n\n75 Points,2004,Red,Mad Dogs & English Men,Red Blend,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,bottlestopper,yuck! Old and well past its best,https://www.cellartracker.com/notes.asp?iWine=...,NaN,bottlestopper wrote:,7/10/2011 - \nbottlestopper wrote:,2003,Red,Mad Dogs & English Men,Red Blend,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19999,Slipknot,whatever,https://www.cellartracker.com/notes.asp?iWine=...,80.0,Slipknot wrote:,11/15/2006 - \nSlipknot wrote: \n\n80 Points,2003,Red,Mad Dogs & English Men,Red Blend,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
